# 1. Content-based recommendation

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('movies.csv')
df.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [3]:
# Adding year column
df[['name', 'year']] = df.title.str.split('\(|\)', expand=True).iloc[:, [0,1]]
df.head(3)

,movieId,title,genres,name,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995


In [4]:
df.drop('title', axis=1, inplace=True)
df.head(3)

,movieId,genres,name,year
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Adventure|Children|Fantasy,Jumanji,1995
2,3,Comedy|Romance,Grumpier Old Men,1995


In [5]:
df['name'] = df['name'].str.strip()

In [6]:
movie_genre_df = df[['genres', 'name']]
movie_genre_df

,genres,name
0,Adventure|Animation|Children|Comedy|Fantasy,Toy Story
1,Adventure|Children|Fantasy,Jumanji
2,Comedy|Romance,Grumpier Old Men
3,Comedy|Drama|Romance,Waiting to Exhale
4,Comedy,Father of the Bride Part II
...,...,...
9737,Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic
9738,Animation|Comedy|Fantasy,No Game No Life: Zero
9739,Drama,Flint
9740,Action|Animation,Bungo Stray Dogs: Dead Apple


In [7]:
movie_genre_df = movie_genre_df.apply(lambda x: x.str.split('|').explode()).reset_index()
movie_genre_df

,index,genres,name
0,0,Adventure,Toy Story
1,0,Animation,Toy Story
2,0,Children,Toy Story
3,0,Comedy,Toy Story
4,0,Fantasy,Toy Story
...,...,...,...
22079,9738,Fantasy,No Game No Life: Zero
22080,9739,Drama,Flint
22081,9740,Action,Bungo Stray Dogs: Dead Apple
22082,9740,Animation,Bungo Stray Dogs: Dead Apple


In [8]:
movie_genre_df.drop('index', inplace=True, axis=1)

In [9]:
movie_genre_df

,genres,name
0,Adventure,Toy Story
1,Animation,Toy Story
2,Children,Toy Story
3,Comedy,Toy Story
4,Fantasy,Toy Story
...,...,...
22079,Fantasy,No Game No Life: Zero
22080,Drama,Flint
22081,Action,Bungo Stray Dogs: Dead Apple
22082,Animation,Bungo Stray Dogs: Dead Apple


In [10]:
movie_genre_df['genres'].unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'War', 'Musical', 'Documentary', 'IMAX',
       'Western', 'Film-Noir', '(no genres listed)'], dtype=object)

In [11]:
# Saving data frame into csv
movie_genre_df.to_csv('movie_genre_df.csv')

##  ✏️ Exercises

> ## 1. Creating content-based data

As much as you might want to jump right to finding similar items and making recommendations, you first need to get your data in a usable format. In the next few exercises, you will explore your base data and work through how to format that data to be used for content-based recommendations.

As a reminder, the desired outcome is a row per movie with each column indicating whether a genre applies to the movie. You will be looking at `movie_genre_df`, which contains these columns:

- `name` - Name of movie
- `genre_list` - Genre that the movie has been labeled as

A movie may have multiple genres, and therefore multiple rows. In this exercise, you will particularly focus on one movie (Toy Story in this case) to be able to clearly see what is happening with the data.

### How many different movies are contained in `movie_genre_df`?


In [12]:
movie_genre_df = pd.read_csv('movie_genre_df.csv', index_col=False)

In [13]:
len(movie_genre_df['name'].unique())

9412

### Get the rows in `movie_genre_df` which have a name equal to Toy Story and save this as `toy_story_genres`.

In [14]:
toy_story_genres = movie_genre_df[movie_genre_df['name'] == "Toy Story"]
toy_story_genres.head()

,Unnamed: 0,genres,name
0,0,Adventure,Toy Story
1,1,Animation,Toy Story
2,2,Children,Toy Story
3,3,Comedy,Toy Story
4,4,Fantasy,Toy Story


### Transform movie_genre_df to a table called movie_cross_table.

Assign the subset of `movie_cross_table` that contains Toy Story to the variable `toy_story_genres_ct` and inspect the results

In [15]:
# Select only the rows with values in the name column equal to Toy Story
toy_story_genres = movie_genre_df[movie_genre_df['name'] == 'Toy Story']

# Create cross-tabulated DataFrame from name and genre_list columns
movie_cross_table = pd.crosstab(movie_genre_df['name'], movie_genre_df['genres'])

# Select only the rows with Toy Story as the index
toy_story_genres_ct = movie_cross_table[movie_cross_table.index == 'Toy Story']
print(toy_story_genres_ct)

genres     (no genres listed)  Action  Adventure  Animation  Children  Comedy  \
name                                                                            
Toy Story                   0       0          1          1         1       1   

genres     Crime  Documentary  Drama  Fantasy  Film-Noir  Horror  IMAX  \
name                                                                     
Toy Story      0            0      0        1          0       0     0   

genres     Musical  Mystery  Romance  Sci-Fi  Thriller  War  Western  
name                                                                  
Toy Story        0        0        0       0         0    0        0  


### Understanding the content-based data

You are now able to convert common attribute data to a DataFrame containing a row per movie, and each of its attributes as columns. You will now take a closer look at the full DataFrame you just created to see if you understand the information within.

A subset of the DataFrame you have created in the last exercise has been loaded as movie_cross_table. As a reminder, the genres are stored as individual columns and the movie names are stored as the index.

Inspect the rows corresponding to 'Toy Story' and 'Yogi Bear' in movie_cross_table. How many genres do they have in common?


Possible Answers

- 0 genres in common

- 2 genres in common ✅ (*Children and comedy*)

- 4 genres in common

- 6 genres in common



In [16]:
movie_cross_table

genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
name,,,,,,,,,,,,,,,,,,,,
'71,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
'Hellboy': The Seeds of Creation,0,1,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0
'Round Midnight,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
'Salem's Lot,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0
'Til There Was You,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
xXx,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
xXx: State of the Union,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0


In [17]:
selected_movies = ["Toy Story", "Yogi Bear"]
movie_cross_table[movie_cross_table.index.isin(selected_movies)]


genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
name,,,,,,,,,,,,,,,,,,,,
Toy Story,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
Yogi Bear,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


> Correct! Yogi Bear and Toy Story both have the 'Children' and 'Comedy' attributes. The more genres that two movies have in common, the more likely it is that someone who liked one will like the other, so now we're going to apply this at a larger scale instead of just one pair of movies.

# Making content-based recommendations

- We need a way to calculate similarities between rows.
- We are going to use the **Jaccard similarity**

$$
J(A, B) = \frac{A \cap B}{A \cup B}
$$

it varies from zero to one. The bigger the jaccard similarity the similar are the items. 

In [18]:
# Data 
movie_genre_df.head()

,Unnamed: 0,genres,name
0,0,Adventure,Toy Story
1,1,Animation,Toy Story
2,2,Children,Toy Story
3,3,Comedy,Toy Story
4,4,Fantasy,Toy Story


In [19]:
# Calculating Jaccard similarity between movies 
from sklearn.metrics import jaccard_score

toy_row = movie_cross_table.loc['Toy Story']
yogi_row = movie_cross_table.loc['Yogi Bear']

In [20]:
print(jaccard_score(toy_row, yogi_row))

0.4


## ✏️Exercises

> # 1. Comparing individual movies with Jaccard similarity

In the last lesson, you built a DataFrame of movies, where each column represents a different genre. You can now use this DataFrame to compare movies by measuring the Jaccard similarity between rows. The higher the Jaccard similarity score, the more similar the two items are.

In this exercise, you will compare the movie GoldenEye with the movie Toy Story, and GoldenEye with SkyFall and compare the results.

The DataFrame movie_cross_table containing all the movies as rows and the genres as Boolean columns that you created in the last lesson has been loaded.

1. Import the Jaccard similarity score function from `sklearn.metrics`.
2. Convert the rows containing 'GoldenEye' and 'Toy Story' to numpy arrays and measure their similarity.
3. Convert the row containing Skyfall to a numpy array and measure its similarity to GoldenEye.

In [21]:
# Data
movie_cross_table.head()

genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
name,,,,,,,,,,,,,,,,,,,,
'71,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
'Hellboy': The Seeds of Creation,0,1,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0
'Round Midnight,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
'Salem's Lot,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0
'Til There Was You,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [22]:
# Import numpy and the Jaccard similarity function
import numpy as np
from sklearn.metrics import jaccard_score

In [23]:
# Extract just the rows containing GoldenEye and Toy Story
goldeneye_values = movie_cross_table.loc['GoldenEye'].values
toy_story_values = movie_cross_table.loc['Toy Story'].values

# Find the similarity between GoldenEye and Toy Story
print(jaccard_score(goldeneye_values, toy_story_values))

0.14285714285714285


In [24]:
# Repeat for GoldenEye and Skyfall
skyfall_values = movie_cross_table.loc['Skyfall'].values
print(jaccard_score(goldeneye_values, skyfall_values))

0.75


> *As you can see, based on Jaccard similarity, GoldenEye and Skyfall (both James Bond movies) are more similar than GoldenEye and Toy Story (a spy movie and an animated kids movie).*

> # 2.Comparing all your movies at once

While finding the Jaccard similarity between any two individual movies in your dataset is great for small-scale analyses, it can prove slow on larger datasets to make recommendations.

In this exercise, you will find the similarities between all movies and store them in a DataFrame for quick and easy lookup.

When finding the similarities between the rows in a DataFrame, you could run through all pairs and calculate them individually, but it's more efficient to use the `pdist()` (pairwise distance) function from `scipy`.

This can be reshaped into the desired rectangular shape using `squareform()` from the same library. Since you want similarity values as opposed to distances, you should subtract the values from 1.

1. Find the Jaccard distance measures between all movies and assign the results to `jaccard_similarity_array`.
2. Create a DataFrame from the `jaccard_similarity_array` with `movie_genre_df.index` as its rows and columns.
3. Print the top 5 rows of the DataFrame and examine the similarity scores.


In [25]:
# Import functions from scipy
from scipy.spatial.distance import pdist, squareform

# Calculate all pairwise distances
jaccard_distances = pdist(movie_cross_table.values, metric='jaccard')

# Convert the distances to a square matrix
jaccard_similarity_array = 1 - squareform(jaccard_distances)

# Wrap the array in a pandas DataFrame
jaccard_similarity_df = pd.DataFrame(jaccard_similarity_array, index=movie_cross_table.index, columns=movie_cross_table.index)

In [26]:
# Print the top 5 rows of the DataFrame
jaccard_similarity_df.head()

name,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,*batteries not included,...All the Marbles,...,Zulu,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté
name,,,,,,,,,,,,,,,,,,,,,
'71,1.000000,0.125,0.200000,0.333333,0.200000,0.0,0.0,0.25,0.000000,0.200000,...,0.600000,0.40,0.2,0.2,0.200000,0.400000,0.400000,0.400000,0.000000,0.000000
'Hellboy': The Seeds of Creation,0.125000,1.000,0.000000,0.000000,0.000000,0.0,0.2,0.00,0.285714,0.166667,...,0.111111,0.00,0.0,0.0,0.000000,0.142857,0.142857,0.142857,0.166667,0.166667
'Round Midnight,0.200000,0.000,1.000000,0.200000,0.333333,0.0,0.0,0.50,0.000000,0.333333,...,0.000000,0.25,0.0,0.0,0.333333,0.000000,0.000000,0.000000,0.000000,0.333333
'Salem's Lot,0.333333,0.000,0.200000,1.000000,0.200000,0.0,0.0,0.25,0.000000,0.200000,...,0.142857,0.75,0.5,0.5,0.200000,0.166667,0.166667,0.166667,0.000000,0.000000
'Til There Was You,0.200000,0.000,0.333333,0.200000,1.000000,0.5,0.0,0.50,0.000000,0.333333,...,0.000000,0.25,0.0,0.0,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000


> # 3. Making recommendations based on movie genres

Now that you have your data in a usable format and know how to compare two movies, the next step is to use this to generate recommendations. In this exercise, you will learn how to generate recommendations for any movie in your dataset. The similarity scores between all movies in the dataset that you calculated in the last exercise have been pre-loaded for you as `jaccard_similarity_array`. `movie_cross_table` containing the movies and their attributes is also available.

For ease of use, you will need to wrap the similarity scores in a DataFrame. Then you will use this new DataFrame to suggest a movie recommendation.


- Generate a DataFrame called jaccard_similarity_df from jaccard_similarity_array.
- Store the similarity values between Thor and all other movies as a Series.
- Sort these from largest to smallest in ordered_similarities.

In [27]:
# Wrap the preloaded array in a DataFrame
jaccard_similarity_df = pd.DataFrame(jaccard_similarity_array, index=movie_cross_table.index, columns=movie_cross_table.index)

# Find the values for the movie Thor
jaccard_similarity_series = jaccard_similarity_df.loc['Thor']

# Sort these values from highest to lowest
ordered_similarities = jaccard_similarity_series.sort_values(ascending=False)

# Print the results
print(ordered_similarities)

name
Thor                                            1.000000
Harry Potter and the Deathly Hallows: Part 2    0.833333
In the Name of the King III                     0.800000
Thor: The Dark World                            0.800000
Seeker: The Dark Is Rising, The                 0.800000
                                                  ...   
Runaway Bride                                   0.000000
Heidi Fleiss: Hollywood Madam                   0.000000
Hedgehog in the Fog                             0.000000
Heavyweights                                    0.000000
À nous la liberté                               0.000000
Name: Thor, Length: 9411, dtype: float64
